# Compartment Reports

SONATA's compartment reports are a hdf5-based format for storing cell time-series variables, like membrane potential or calcium concentration, for multi-compartment or point (eg single compartment) neurons. With an emphasis on recording values from large-scale simulations that involves hundreds to millions of indivdiual cells.


## Recording compartment reports

The pySonata reports module includes code for writing (and reading) SONATA compliant files. To start with we must create a CompartmentReport object with the name of the final file that will be generated (**output/membrane_potentials.h5**). As the name suggests we are simulating the recording of membrane potential (variable=mV) in units of mV. We assume that we are recording at uniform intervals at 0.1 ms for 100.0 ms total time.

In [1]:
import numpy as np

from sonata.reports.compartment import CompartmentReport

In [2]:
cr = CompartmentReport('output/membrane_potentials.h5', mode='w', tstart=0.0, tstop=100.0, dt=0.1, 
                       units='mV', variable='mV')

Before we start recording the membrane potential we must specify what nodes (eg cells) will be part of the recording. Every node is identified by a unique node_id and the population (in this case Visual Area 1) it belongs  to. 

A single cell has multiple points of recordings, which is identified by the element (eg segment) id and the position along the element being recorded from. Here was are setting:
 * node 0 has 100 segments, we are recording from the center of each one.
 * node 1 is a point neuron
 * node 2 may have multiple segments, but we are only recording from 3 equidistant points in segment 0

In [3]:
cr.add_cell(node_id=0, element_ids=range(100), element_pos=[0.5]*100, population='V1')
cr.add_cell(node_id=1, element_ids=[0], element_pos=[0.0], population='V1')
cr.add_cell(node_id=2, element_ids=[0, 0, 0], element_pos=[0.25, 0.5, 0.75], population='V1')
cr.initialize()

Next step is to actually record the variable across all nodes/segment. You can do that incrementally at each time-step:

In [4]:
for t in range(0, 1000):
    cr.record_cell(node_id=0, population='V1', vals=[-25.5]*100, tstep=t)
    cr.record_cell(node_id=1, population='V1', vals=-60+t*0.6, tstep=t)

or record a large block of times

In [5]:
cr.record_cell_block(node_id=2, population='V1', vals=np.random.uniform(-55.0, 10.0, size=(1000, 3)), 
                     beg_step=0, end_step=999)

In [6]:
cr.close()

## Extras

### Setting a default population

Often there is only a single population of nodes being recorded from, which makes specifcying the *population* parameter during each call to add_cell redundant and cumbersome. In this case use the *default_population* parameter when initalizing the report:

```python
cr = Compartment_report('output/membrane_potentials.h5, mode='w, default_population='V1', ...)
cr.add_cell(node_id=0, element_ids=range(100), element_pos=[0.5]*100)
...
```


### Non-uniform recording

By default the sonata format assumes that the variables are being recorded at uniform intervals, which allows the time-trace to be stored and represented using only three values: *tstart*, *tstop*, *dt*. If the time trace is non-uniform then you can use the set_time_trace() method instead:

```python
cr = Compartment_report('output/membrane_potentials.h5, mode='w, default_population='V1', n_steps=1000)
cr.set_time_trace(np.logspace(0.0, 2.0, num=1000))
...
```

### Parallelized Recordings

The API is designed to work with MPI when running a simulation across multiple cores, assuming different cells (or different elements of a cell) are divided up among each core. 

```python
from mpi4py import MPI
rank = MPI.COMM_WORLD.comm.Get_rank()

cr = CompartmentReport('output/membrane_potentials.h5', mode='w', tstart=0.0, tstop=100.0, dt=0.1, 
                       units='mV', variable='mV')

if rank == 0:
    cr.add_cell(node_id=0, element_ids=range(100), element_pos=[0.5]*100, population='V1')
    cr.initalize()
    cr.record_cell_block(node_id=0, population='V1', vals=np.random.uniform(-55.0, 10.0, size=(1000, 100)),
                         beg_step=0, end_step=999)
elif rank == 1:
    cr.add_cell(node_id=1, element_ids=[0], element_pos=[0.0], population='V1')
    cr.initalize()
    cr.record_cell_block(node_id=1, population='V1', vals=np.random.uniform(-55.0, 10.0, size=1000), 
                         beg_step=0, end_step=999)

cr.close()

```




## Reading

Now that we have generated (or given) a compartmental report we want to be able to read it into our code. To do so make sure to open the hdf5 file in read (**r**) mode:

In [7]:
cr = CompartmentReport('output/membrane_potentials.h5', mode='r')

A report may contain multiple populations of nodes. In the above case there was only one (V1):

In [8]:
print('populations: {}'.format(cr.populations))
print('V1')
v1_voltages = cr['V1']
print(' node ids: {}'.format(v1_voltages.node_ids()))
print(' variable: {}'.format(v1_voltages.units()))
print(' units: {}'.format(v1_voltages.units()))

populations: ['V1']
V1
 node ids: [0 1 2]
 variable: mV
 units: mV


We can also get the time-trace for the simulation as a numpy array.

In [9]:
print('simulation steps: {}'.format(cr.time_trace('V1')))


simulation steps: [  0.           0.1001001    0.2002002    0.3003003    0.4004004
   0.5005005    0.6006006    0.7007007    0.8008008    0.9009009
   1.001001     1.1011011    1.2012012    1.3013013    1.4014014
   1.5015015    1.6016016    1.7017017    1.8018018    1.9019019
   2.002002     2.1021021    2.2022022    2.3023023    2.4024024
   2.5025025    2.6026026    2.7027027    2.8028028    2.9029029
   3.003003     3.1031031    3.2032032    3.3033033    3.4034034
   3.5035035    3.6036036    3.7037037    3.8038038    3.9039039
   4.004004     4.1041041    4.2042042    4.3043043    4.4044044
   4.5045045    4.6046046    4.7047047    4.8048048    4.9049049
   5.00500501   5.10510511   5.20520521   5.30530531   5.40540541
   5.50550551   5.60560561   5.70570571   5.80580581   5.90590591
   6.00600601   6.10610611   6.20620621   6.30630631   6.40640641
   6.50650651   6.60660661   6.70670671   6.80680681   6.90690691
   7.00700701   7.10710711   7.20720721   7.30730731   7.40740741
  

Use the **data()** method to get the recorded values for each cell. 

For a point neuron the **data** method returns a vector:

In [10]:
v1_voltages.data(node_id=1)

array([[-60. ],
       [-59.4],
       [-58.8],
       [-58.2],
       [-57.6],
       [-57. ],
       [-56.4],
       [-55.8],
       [-55.2],
       [-54.6],
       [-54. ],
       [-53.4],
       [-52.8],
       [-52.2],
       [-51.6],
       [-51. ],
       [-50.4],
       [-49.8],
       [-49.2],
       [-48.6],
       [-48. ],
       [-47.4],
       [-46.8],
       [-46.2],
       [-45.6],
       [-45. ],
       [-44.4],
       [-43.8],
       [-43.2],
       [-42.6],
       [-42. ],
       [-41.4],
       [-40.8],
       [-40.2],
       [-39.6],
       [-39. ],
       [-38.4],
       [-37.8],
       [-37.2],
       [-36.6],
       [-36. ],
       [-35.4],
       [-34.8],
       [-34.2],
       [-33.6],
       [-33. ],
       [-32.4],
       [-31.8],
       [-31.2],
       [-30.6],
       [-30. ],
       [-29.4],
       [-28.8],
       [-28.2],
       [-27.6],
       [-27. ],
       [-26.4],
       [-25.8],
       [-25.2],
       [-24.6],
       [-24. ],
       [-23.4],
       [

When there are multiple recordings a matrix is returned, where the rows indicate a time-point and each column a different segement/position. You can use the **element_ids** and **element_pos** methods to map each column.

In [11]:
print('element_ids: \n{}'.format(v1_voltages.element_ids(node_id=2)))
print('element_pos: \n{}'.format(v1_voltages.element_pos(node_id=2)))
print('data: \n{}'.format(v1_voltages.data(node_id=2)))

element_ids: 
[0 0 0]
element_pos: 
[0.25 0.5  0.75]
data: 
[[-26.68004873 -44.57869137 -36.35318618]
 [-27.65466567 -30.37087527 -31.90339667]
 [-45.72573791  -7.59687698  -6.1168696 ]
 ...
 [-25.30565698 -40.37708363   8.16683852]
 [-42.98827417 -10.66182614 -45.61760536]
 [-47.5588135  -52.18320142  -8.49817789]]
